# 📚 Notebook: Modelo compacto (español) entrenado con Wikipedia
Este notebook está listo para ejecutar en Google Colab. Incluye:

- Descarga de un pequeño subconjunto de Wikipedia en español
- Preprocesamiento y tokenización
- Fine-tuning de `distilgpt2` con correcciones (pad token, resize embeddings)
- Parámetros de generación que reducen repeticiones
- Interfaz Gradio con sliders para ajustar generación en tiempo real

⚠️ Nota: ejecutar el entrenamiento puede tardar y consumir recursos. Si usas Colab gratuito, ajusta los parámetros para reducir el coste.

In [ ]:
!pip install -q transformers datasets gradio wikipedia torch --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import re
import wikipedia
import gradio as gr
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [ ]:
def recolectar_wikipedia_es(subset_percent='0.05%'):
    ds = load_dataset('wikipedia', '20220301.es', split=f"train[:{subset_percent}]")
    textos = [t for t in ds['text'] if (t and len(t.strip())>200)]
    return textos

def limpiar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'\[[^\]]*\]', '', texto)
    texto = re.sub(r'==.*?==', '', texto)
    return texto.strip()

In [ ]:
print('Descargando un subconjunto pequeño de Wikipedia (es) — puede tardar unos minutos...')
textos = recolectar_wikipedia_es('0.05%')
print(f'Se descargaron {len(textos)} páginas (filtradas).')
textos_limpios = [limpiar_texto(t) for t in textos]
N = 200
textos_entrenamiento = textos_limpios[:N]
print(f'Usando {len(textos_entrenamiento)} textos para entrenamiento (puedes cambiar N).')

In [ ]:
modelo_id = 'distilgpt2'
print('Cargando tokenizer y modelo...')
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
modelo = AutoModelForCausalLM.from_pretrained(modelo_id)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    modelo.resize_token_embeddings(len(tokenizer))

tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
modelo.config.pad_token_id = modelo.config.get('pad_token_id', tokenizer.convert_tokens_to_ids(tokenizer.pad_token))
print('Tokenizador y modelo listos. Longitud vocab:', len(tokenizer))

In [ ]:
raw_ds = Dataset.from_dict({'text': textos_entrenamiento})
max_length = 128
def tokenizar_batch(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=max_length)
print('Tokenizando...')
dataset_tokenizado = raw_ds.map(tokenizar_batch, batched=True, remove_columns=['text'])
print(dataset_tokenizado)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir='./modelo_compacto_es',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    remove_unused_columns=False
)
trainer = Trainer(
    model=modelo,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_tokenizado
)
print('Listo para entrenar. Ejecuta trainer.train() para iniciar el fine-tuning (puede tardar).')

In [ ]:
# trainer.train()
# trainer.save_model('./modelo_compacto_es')
print('Si quieres entrenar, descomenta trainer.train() y trainer.save_model(...)')

In [ ]:
import torch
from typing import Dict

def generar_texto(prompt: str, max_length: int=80, temperature: float=1.0, top_p: float=0.92, top_k: int=50, repetition_penalty: float=1.8, no_repeat_ngram_size: int=3):
    modelo.eval()
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors='pt')
        output = modelo.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=True,
            num_return_sequences=1
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
iface = gr.Interface(
    fn=generar_texto,
    inputs=[
        gr.Textbox(lines=2, label='Escribe una frase inicial (prompt)'),
        gr.Slider(minimum=20, maximum=200, step=10, value=80, label='max_length'),
        gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.0, label='temperature'),
        gr.Slider(minimum=0.1, maximum=1.0, step=0.01, value=0.92, label='top_p'),
        gr.Slider(minimum=1, maximum=200, step=1, value=50, label='top_k'),
        gr.Slider(minimum=1.0, maximum=3.0, step=0.1, value=1.8, label='repetition_penalty'),
        gr.Slider(minimum=1, maximum=5, step=1, value=3, label='no_repeat_ngram_size')
    ],
    outputs=gr.Textbox(label='Texto generado'),
    title='🧠 Modelo compacto (es) - prueba interactiva',
    description='Ajusta los sliders para controlar la generación y evitar repeticiones.'
)

iface.launch(enable_flagging=False)
